In [3]:
import pandas as pd
from rapidfuzz import process, fuzz
import src as src
pd.set_option('display.max_columns', None)

In [2]:
df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")

C:\Users\pepel\AppData\Local\Temp\ipykernel_76340\3719518088.py:1: DtypeWarning: Columns (0,1,3,11,12,19,34,37,39,40,41,42,43,44,45,46,47,49,50,51,52,53,55,56,57,58,59,60,71,73,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")


In [3]:
df_lesiones = pd.read_csv("data/lesiones_completo.csv")

In [4]:
df_jugadoresunicos_eventos = df_eventos["player"].unique().tolist()
df_jugadoresunicos = df_lesiones["href"].unique().tolist()
df_clubesunicos_eventos = df_eventos["team"].unique().tolist()
df_clubesunicos = df_lesiones["equipo"].unique().tolist()

In [5]:
df_nombres = src.normalizacion_nombres(df_jugadoresunicos, df_jugadoresunicos_eventos)

NameError: name 'df_jugadoresunicos' is not defined

In [ ]:
df_clubes = src.normalizacion_nombres(df_clubesunicos, df_clubesunicos_eventos)

In [7]:
df_nombres[df_nombres["Nombre_unificado"].isnull()]

,Nombre_original,Nombre_unificado
193,Yroundou Musavu-King,None
205,Ousseynou Cissé,None


In [8]:
dict_nombres_info = df_nombres.set_index('Nombre_original')['Nombre_unificado'].to_dict()

In [9]:
df_lesiones["href"] = df_lesiones["href"].apply(
    lambda x: dict_nombres_info.get(x, x) if dict_nombres_info.get(x) is not None else x
)

In [10]:
# Crear diccionario para mapear los equipos
df_equipos_jugadores = df_eventos[["player", "team"]].drop_duplicates()
dict_equipos_jugadores = dict(zip(df_equipos_jugadores["player"], df_equipos_jugadores["team"]))
# Mapear los equipos en la temporada 2015/2016 ya que la info que aparece es actual
df_lesiones["equipo"] = df_lesiones["href"].map(dict_equipos_jugadores)

In [11]:
# Crear diccionarios para mapear los IDs de jugador y equipo
df_equipos = df_eventos[["team", "team_id"]].drop_duplicates()
dict_equipos = dict(zip(df_equipos["team"], df_equipos["team_id"]))
df_jugadores = df_eventos[["player", "player_id"]].drop_duplicates()
dict_jugadores = dict(zip(df_jugadores["player"], df_jugadores["player_id"]))

In [12]:
# Mapear ID Jugadores
df_lesiones["playerID"] = df_lesiones["href"].map(dict_jugadores)
# Mapear ID Equipos
df_lesiones["clubID"] = df_lesiones["equipo"].map(dict_equipos)

In [13]:
tipo_lesiones_unico = df_lesiones["TIPO"].unique().tolist()
subtipo_lesiones_unico = df_lesiones["SUB TIPO"].unique().tolist()

In [14]:
dict_subtipo = {}
item = 0
for x in subtipo_lesiones_unico:
    item = item + 1
    dict_subtipo[x] = item

dict_tipo = {}
item_1 = 1
for x in tipo_lesiones_unico:
    item_1 = item_1 + 1
    dict_tipo[x] = item_1

In [15]:
df_lesiones["TIPO_ID"] = df_lesiones["TIPO"].map(dict_tipo)
df_lesiones["SUB_TIPO_ID"] = df_lesiones["SUB TIPO"].map(dict_subtipo)

In [16]:
df_lesiones.to_csv("data/lesiones_completo_normalizado.csv", index=False)